# Test Full Code with Birmingham
### Gavin Rolls

This notebook will be a compilation of the methods I've applied to London as my area of study and attempt to recreate the model in Birmingham

# Data Collection

### Imports + Database Configuration

In [2]:
#Library Imports - using DuckDB for Overture Import

#Database
import duckdb

#Basics
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim

#OpenStreetMap
import osmnx as ox

In [3]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [6]:
%%sql      
INSTALL httpfs;

LOAD httpfs;

INSTALL spatial;

LOAD spatial;

SET s3_region='us-west-2';

,Success


### Download Buildings - Overture

Downloaded as 'data/bham_buildings_overture.geojson'

In [12]:
#Get Birmingham Bounding Box

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get location data
location = geolocator.geocode("Birmingham")

# Get the bounding box
bounding_box = location.raw['boundingbox']

# Convert bounding box to coordinates
min_lat, max_lat = float(bounding_box[0]), float(bounding_box[1])
min_lon, max_lon = float(bounding_box[2]), float(bounding_box[3])

print(min_lon)
print(min_lat)
print(max_lon)
print(max_lat)

-2.0336486
52.381053
-1.7288417
52.6087058


In [ ]:
%%sql

LOAD azure;

SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=overturemapswestus2;AccountKey=;EndpointSuffix=core.windows.net';
COPY (
SELECT
    names.primary as primary_name,
    height,
    level,
    ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('azure://release/2024-05-16-beta.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE primary_name IS NOT NULL
AND bbox.xmin > -2.0336486
AND bbox.xmax < -1.7288417
AND bbox.ymin > 52.381053
AND bbox.ymax < 52.6087058
) TO 'data/overture_data/bham_buildings_overture.geojson'
WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');


### Download POIs - Overture

Downloaded as 'data/bham_places_overture.geojson''

In [ ]:
%%sql

COPY (
    SELECT
        names.primary AS name,
        categories.main as category,
        ROUND(confidence,2) as confidence,
        ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/*/*')
WHERE
    bbox.xmin BETWEEN -2.0336486 AND -1.7288417 AND
    bbox.ymin BETWEEN 52.381053 AND 52.6087058
) TO 'data/overture_data/bham_places_overture.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

### Overture Download Stats

In [ ]:
#Basic overview stats of Birmingham Data - Using the locally saved files here
buildings = gpd.read_file('data/overture_data/bham_buildings_overture.geojson')
places = gpd.read_file('data/overture_data/bham_places_overture.geojson')

#Count of Features
print("Birmingham Building Count: " + str(buildings.shape[0]))
print("Birmingham POI count: " + str(places.shape[0]))

### Download Buildings - OSM

Downloaded as 'data/osmbuildings/building_footprints.geojson'

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

#Test new config
ox.config(use_cache=True, log_console=True)

buildings = ox.features_from_place(place_name, tags={'building': True})
buildings = buildings[buildings.geometry.notnull()]
building_footprints = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints.columns:
    if building_footprints.apply(lambda x: isinstance(x, list)).any():
        building_footprints = building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints = building_footprints[['name', 'geometry']].reset_index()
print(building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints.to_file("data/osm_data/bham_buildings.geojson", driver="GeoJSON")

### Extract Building Type Info from OSM - Commercial

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

commercial_buildings = ox.features_from_place(place_name, tags={'building': ['commercial']})

commercial_buildings = commercial_buildings[commercial_buildings.geometry.notnull()]
commercial_building_footprints = commercial_buildings[commercial_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints.columns:
    if commercial_building_footprints.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints = commercial_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints = commercial_building_footprints[['name', 'geometry']].reset_index()
print(commercial_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints.to_file("data/osm_data/bham_commercial_buildings.geojson", driver="GeoJSON")

### Extract Building Type Info from OSM - Office

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

office_buildings = ox.features_from_place(place_name, tags={'building': ['office']})

office_buildings = office_buildings[office_buildings.geometry.notnull()]
office_building_footprints = office_buildings[office_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints.columns:
    if office_building_footprints.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints[col] = office_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints = office_building_footprints[['name', 'geometry']].reset_index()
print(office_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints.to_file("data/osm_data/bham_office_buildings.geojson", driver="GeoJSON")


### Extract Building Type Info from OSM - Residential

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

residential_buildings = ox.features_from_place(place_name, tags={'building': ['residential']})

residential_buildings = residential_buildings[residential_buildings.geometry.notnull()]
residential_building_footprints = residential_buildings[residential_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints.columns:
    if residential_building_footprints.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints[col] = residential_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints = residential_building_footprints[['name', 'geometry']].reset_index()
print(residential_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints.to_file("data/osm_data/bham_residential_buildings.geojson", driver="GeoJSON")


### Extract Building Type Info from OSM - Retail

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

retail_buildings = ox.features_from_place(place_name, tags={'building': ['retail']})

retail_buildings = retail_buildings[retail_buildings.geometry.notnull()]
retail_building_footprints = retail_buildings[retail_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints.columns:
    if retail_building_footprints.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints[col] = retail_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints = retail_building_footprints[['name', 'geometry']].reset_index()
print(retail_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints.to_file("data/osm_data/bham_retail_buildings.geojson", driver="GeoJSON")


# Data Cleaning
### Taken from v2